## Basic querying of PO



In [3]:
## Create an ontology factory and use it to make an ontology object
from ontobio.ontol_factory import OntologyFactory
ofa = OntologyFactory()
po = ofa.create('po')

In [5]:
## Node ids matching leaflet
po.search('%leaflet%')

['PO:0025362', 'PO:0006034', 'PO:0025481', 'PO:0020049', 'PO:0004500']

In [7]:
## Write id-labels for classes with a label matching 'leaflet'
terms = po.search('%leaflet%')
for t in terms:
    print(" Match: {id} '{label}'".format(id=t, label=po.label(t)))

 Match: PO:0025362 'leaflet tendril'
 Match: PO:0006034 'leaflet margin'
 Match: PO:0025481 'leaflet primordium'
 Match: PO:0020049 'leaflet'
 Match: PO:0004500 'formation of vascular leaflet primordia stage'


In [8]:
## Get the PO node for 'leaflet'
[leaflet] = po.search('leaflet')

In [9]:
leaflet

'PO:0020049'

In [10]:
## Fetch descendants (all relations)
po.descendants(leaflet)

{'PO:0006034', 'PO:0020050', 'PO:0020051', 'PO:0025362'}

In [11]:
## Write descendants
for t in po.descendants(leaflet):
    print(" Descendant: {id} '{label}'".format(id=t, label=po.label(t)))

 Descendant: PO:0020051 'stipel'
 Descendant: PO:0006034 'leaflet margin'
 Descendant: PO:0020050 'petiolule'
 Descendant: PO:0025362 'leaflet tendril'


In [12]:
## Ancestors of leaflet (all relations)
for t in po.ancestors(leaflet):
    print(" Anc: {id} '{label}'".format(id=t, label=po.label(t)))

 Anc: NCBITaxon:2759 'Eukaryota'
 Anc: PO:0009007 'portion of plant tissue'
 Anc: PO:0025034 'leaf'
 Anc: PO:0000225 'peripheral zone'
 Anc: PO:0025606 'native plant cell'
 Anc: PO:0000017 'vascular leaf primordium'
 Anc: PO:0007033 'whole plant development stage'
 Anc: PO:0025131 'plant anatomical entity'
 Anc: PO:0025496 'multi-tissue plant structure'
 Anc: PO:0000003 'whole plant'
 Anc: PO:0009025 'vascular leaf'
 Anc: PO:0025128 'phyllome primordium'
 Anc: PO:0000037 'shoot apex'
 Anc: PO:0025029 'shoot axis'
 Anc: PO:0009011 'plant structure'
 Anc: PO:0006079 'shoot system meristem'
 Anc: PO:0025001 'cardinal organ part'
 Anc: NCBITaxon:131567 'cellular organisms'
 Anc: PO:0025127 'primordium'
 Anc: PO:0025004 'plant axis'
 Anc: PO:0025223 'vegetative shoot apex'
 Anc: PO:0025431 'vascular leaf anlagen'
 Anc: PO:0006210 'protoderm'
 Anc: NCBITaxon:1 'root'
 Anc: PO:0020148 'shoot apical meristem'
 Anc: PO:0008016 'vegetative shoot apical meristem'
 Anc: PO:0009013 'portion of meri

In [13]:
## Ancestors of leaflet (isa/part-of)
for t in po.ancestors(leaflet, relations=['subClassOf', 'BFO:0000050']):
    print(" Anc (isa/po): {id} '{label}'".format(id=t, label=po.label(t)))

 Anc (isa/po): PO:0009011 'plant structure'
 Anc (isa/po): PO:0025497 'collective plant structure'
 Anc (isa/po): PO:0006001 'phyllome'
 Anc (isa/po): PO:0025034 'leaf'
 Anc (isa/po): PO:0020043 'compound leaf'
 Anc (isa/po): PO:0025007 'collective plant organ structure'
 Anc (isa/po): PO:0025498 'cardinal part of multi-tissue plant structure'
 Anc (isa/po): PO:0025001 'cardinal organ part'
 Anc (isa/po): PO:0025131 'plant anatomical entity'
 Anc (isa/po): PO:0025496 'multi-tissue plant structure'
 Anc (isa/po): PO:0009025 'vascular leaf'
 Anc (isa/po): PO:0009006 'shoot system'
 Anc (isa/po): PO:0009008 'plant organ'


In [19]:
## Get all terms in a subset (slim)
po.extract_subset('Angiosperm')

['PO:0000002',
 'PO:0000191',
 'PO:0000289',
 'PO:0001078',
 'PO:0001081',
 'PO:0001094',
 'PO:0001095',
 'PO:0001180',
 'PO:0001185',
 'PO:0004507',
 'PO:0004700',
 'PO:0005360',
 'PO:0006006',
 'PO:0006007',
 'PO:0006008',
 'PO:0006009',
 'PO:0006321',
 'PO:0006324',
 'PO:0006340',
 'PO:0006342',
 'PO:0007010',
 'PO:0007016',
 'PO:0007024',
 'PO:0007026',
 'PO:0007042',
 'PO:0007047',
 'PO:0007052',
 'PO:0007053',
 'PO:0007615',
 'PO:0007616',
 'PO:0007631',
 'PO:0008001',
 'PO:0008002',
 'PO:0008018',
 'PO:0009001',
 'PO:0009005',
 'PO:0009010',
 'PO:0009025',
 'PO:0009032',
 'PO:0009046',
 'PO:0009049',
 'PO:0009052',
 'PO:0009053',
 'PO:0009069',
 'PO:0009070',
 'PO:0009071',
 'PO:0009072',
 'PO:0009081',
 'PO:0009101',
 'PO:0020002',
 'PO:0020004',
 'PO:0020028',
 'PO:0020096',
 'PO:0020097',
 'PO:0020103',
 'PO:0020109',
 'PO:0020121',
 'PO:0020122',
 'PO:0025029',
 'PO:0025074',
 'PO:0025104',
 'PO:0025121',
 'PO:0025123',
 'PO:0025240',
 'PO:0025241',
 'PO:0025281',
 'PO:00253

In [21]:
## Create a sub-ontology
## This sub-ontology matches the selected slim
angiosperm = po.subontology(po.extract_subset('Angiosperm'), relations=['subClassOf', 'BFO:0000050'])

In [23]:
## Write out sub-ontology as ascii-tree
from ontobio.io.ontol_renderers import GraphRenderer
w = GraphRenderer.create('tree')
w.write(angiosperm)

. PO:0009001 ! fruit
 < PO:0008001 ! fruit distal end
 < PO:0008002 ! fruit proximal end

. PO:0007631 ! plant embryo development stage
 % PO:0001078 ! plant embryo cotyledonary stage
 % PO:0001095 ! plant embryo true leaf formation stage
 % PO:0001081 ! mature plant embryo stage
  % PO:0025377 ! plant embryo dormant stage
 % PO:0004507 ! plant embryo bilateral stage
 % PO:0001180 ! plant proembryo stage
 % PO:0001185 ! plant embryo globular stage
 % PO:0001094 ! plant embryo coleoptilar stage

. PO:0025281 ! pollen
 < PO:0025121 ! pollen sperm cell

. PO:0009025 ! vascular leaf
 % PO:0006339 ! juvenile vascular leaf
 % PO:0008018 ! transition vascular leaf
 % PO:0006340 ! adult vascular leaf
  % PO:0020103 ! flag leaf

. PO:0025029 ! shoot axis
 % PO:0020122 ! inflorescence axis
  % PO:0025104 ! first order inflorescence axis
   % PO:0006324 ! first order tassel inflorescence axis
  % PO:0009081 ! inflorescence branch
   % PO:0006321 ! primary inflorescence branch
   % PO:0009052 ! fl

In [24]:
## Write as png
## (requires obographviz from npm)
w = GraphRenderer.create('png')
w.outfile = 'output/angiosperm.png'
w.write(angiosperm)

![img](output/angiosperm.png)